In [7]:
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn
from sklearn.cluster import KMeans
seaborn.set()

In [8]:
def clean_name(archive_list):
    # Adapted From: https://stackoverflow.com/questions/5843518/remove-all-special-characters-punctuation-and-spaces-from-string
    return re.sub('[^A-Za-z0-9]+', '', archive_list["Name"])

In [22]:
df_index = pd.read_csv('./index_data/closest_weather_to_hydro_stations.csv',sep=',')

df_index = df_index.rename(columns={df_index.columns[0]:'hydro_id'})
df_index = df_index[['Station ID', 'Name']]
df_index['temp_name'] = df_index.apply(clean_name,axis=1)
df_index = df_index.drop_duplicates()
df_index

,Station ID,Name,temp_name
0,1423,MACKENZIE A,MACKENZIEA
1,450,SUSKWA VALLEY,SUSKWAVALLEY
3,888,VANCOUVER HARBOUR CS,VANCOUVERHARBOURCS
5,823,N VANC GROUSE MTN RESORT,NVANCGROUSEMTNRESORT
7,698,N VAN SEYMOUR HATCHERY,NVANSEYMOURHATCHERY
11,336,SQUAMISH AIRPORT,SQUAMISHAIRPORT
13,97,SHAWNIGAN LAKE,SHAWNIGANLAKE
14,162,COURTENAY PUNTLEDGE,COURTENAYPUNTLEDGE
15,242,CONUMA RIVER HATCHERY,CONUMARIVERHATCHERY
16,145,CAMPBELL RIVER A,CAMPBELLRIVERA


In [70]:
def cluster_day(filtered_df):
    filtered_df = filtered_df[['Max Temp (°C)','Min Temp (°C)',
                             'Mean Temp (°C)','Total Rain (mm)',
                             'Total Snow (cm)','Total Precip (mm)',
                             'Snow on Grnd (cm)']]
    filtered_df = filtered_df.fillna(0)
    if len(filtered_df) == 0:
        return None
    return KMeans(n_clusters=3).fit(filtered_df).cluster_centers_

def gather_cluster_data(df_list):
    df = pd.read_csv("./data/consolidated_weather_data/{}-daily.csv".format(df_list))
#     df = df.astype(float)
    df = df.drop(['Date/Time'],axis=1)
    df = df.sort_values(['Year', 'Month', 'Day'], ascending=[True, True, True])\
            .groupby(['Month', 'Day'], as_index=False)\
            .fillna(method='ffill')\
            .reset_index(drop=True)
    
    df = df.drop(['Year'],axis=1)
    
    grouped_data = df.groupby(['Month','Day'])
    data_list = []
    for key, item in grouped_data:
        cluster_means = cluster_day(grouped_data.get_group(key))
        for cluster_id in range(3):
            data_list.append([key[0], key[1], cluster_id] + list(cluster_means[cluster_id]))
    
    
    data = pd.DataFrame(data_list)
    rename_map = {data.columns[0]:'month',
                  data.columns[1]:'day',
                  data.columns[2]:'cluster_id',
                  data.columns[3]:'Max Temp (°C)',
                  data.columns[4]:'Min Temp (°C)',
                  data.columns[5]:'Mean Temp (°C)',
                  data.columns[6]:'Total Rain (mm)',
                  data.columns[7]:'Total Snow (cm)',
                  data.columns[8]:'Total Precip (mm)',
                  data.columns[9]:'Snow on Grnd (cm)'
                 }
    data = data.rename(columns=rename_map)
    fpath = "./data/cluster_data/{}-daily.csv".format(df_list)
    data.to_csv(fpath)
    
df_index['Station ID'].apply(gather_cluster_data)

0     None
1     None
3     None
5     None
7     None
11    None
13    None
14    None
15    None
16    None
17    None
18    None
21    None
22    None
23    None
24    None
25    None
27    None
28    None
29    None
31    None
32    None
33    None
35    None
Name: Station ID, dtype: object

In [44]:
# ./data/consolidated_weather_data/{}-monthly.csv
# ./data/consolidated_weather_data/{}-daily.csv

Station ID           1423
Name          MACKENZIE A
temp_name      MACKENZIEA
Name: 0, dtype: object